In [0]:
# Imports of libraries and frameworks:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Set the random seed for reproducibility

seed = 69
np.random.seed(seed)
tf.set_random_seed(seed)

Load dataset:

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
dataset = pd.read_csv('v000_SCIG_SC_SENSORC_FOURIER_010.csv');

In [0]:
dataset.head()

Remove unwanted features

In [0]:
unwanteFeatures = ['idx', 'fx1_R', 'fx1_S', 'fx1_T', 'Freq_Rated', 'Power']
dataset_important_features = dataset.drop(unwanteFeatures, axis=1)

dataset_important_features.head()

In [0]:
# X are the inputs and y the outputs:

X = dataset_important_features.values[:,:-1]
y = dataset_important_features.values[:,-1]

print(y)

In [0]:
import numpy as np
Y = np.array(y)

Y-=1
Y

In [0]:
y = keras.utils.to_categorical(Y, num_classes=7)
print(y)

### Preprocessing data with standardization

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler();
std_scaler.fit(X)

X = std_scaler.transform(X)

## Training with 10-fold Cross Validation, tensorflow and keras

In [0]:
print(tf.__version__)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, shuffle=True)

### Create a keras model

Define, optimzer, loss and mectrics

In [0]:
model = Sequential()

model.add(Dense(10, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(7, activation='softmax'))
          

model.compile(optimizer='adam', 
            loss='categorical_crossentropy', 
            metrics=['accuracy'])

In [0]:
history=model.fit(X_train, y_train, epochs=300, verbose=1,validation_data=(X_test, y_test))

In [0]:
def plot_history(history,show=True):
  

    # summarize history for accuracy
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.grid(True)
    plt.title('accuracy evolution')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')

    if (show): 
      plt.show()
    else: 
      plt.clf()
      
    # summarize history for loss
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.grid(True)
    plt.title('loss evolution')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    
    if (show): 
      plt.show()
    else: 
      plt.clf()

    return True

import seaborn as sns

plot_history(history.history,show=True)

In [0]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test accuracy:', test_acc)

In [0]:
y_hat = np.argmax(model.predict(X_test), axis=1)
y_test = np.argmax(y_test, axis=1)

In [0]:
def percentage_confusion_matrix (mat):
    return np.around(100*(mat / mat.sum(axis=1)[:,None]), 2)

from sklearn.metrics import confusion_matrix

percentage_confusion_matrix(confusion_matrix(y_test, y_hat))